IMPORTS

In [5]:
import pandas as pd 
import matplotlib.pyplot as plt
from FetchData import fetch_data
from Utils import  *

Constants

In [ ]:
DISTANCE_GALCHI_SUIRENITAR=30000
DISTANCE_BUDHI_SUIRENITAR=18500

MATPLLOTLIB SETTINGS

In [15]:
plt.style.use('ggplot')

Preprocessing

In [6]:
def preprocess_data(df):   
    df['datetime']=pd.to_datetime(df['datetime'])
    df['value']=pd.to_numeric(df['value'],errors='coerce')
    df.set_index('datetime',inplace=True)
    # df=handle_outliers(df)
    # handle_null_values(df)
    return df


CONSTANTS

In [7]:
#Getting Data and creating Dataframe.
GalchiSeriesId=7027
SuirentarSeriesID=7009
chumlingatSeriesID=6991
BudhiGandakiSeriesID=7045

START_DATE="2024-12-14:00:00"
END_DATE="2024-12-16T00:00:00"

FETCH FROM API AND CREATE DF

In [8]:
budhi_gandaki = fetch_data(BudhiGandakiSeriesID,START_DATE,END_DATE)
galchi = fetch_data(GalchiSeriesId,START_DATE,END_DATE)
suirenitar = fetch_data(SuirentarSeriesID, "2024-12-14:00:00", "2024-12-22T00:00:00")

#creating DataFrame
budhi_gandaki_df=pd.DataFrame(budhi_gandaki)
galchi_df=pd.DataFrame(galchi)
suirenitar_df=pd.DataFrame(suirenitar)

# #preprocess
budhi_gandaki_df=preprocess_data(budhi_gandaki_df)
galchi_df=preprocess_data(galchi_df)
suirenitar_df=preprocess_data(suirenitar_df)

#export csv
budhi_gandaki_df.to_csv('Dataset/Dev/Original_Fetched/budhi_gandaki_df.csv',index=True)
galchi_df.to_csv('Dataset/Dev/Original_Fetched/galchi_df.csv',index=True)
suirenitar_df.to_csv('Dataset/Dev/Original_Fetched/suirenitar_df.csv',index=True)

Visualizing Data

In [ ]:
budhi_gandaki_df=budhi_gandaki_df[['value']]
galchi_df=galchi_df[['value']]
suirenitar_df=suirenitar_df[['value']]
fig,ax=plt.subplots(3,1,figsize=(20,10),sharex=True)
budhi_gandaki_df.plot(color='red',ax=ax[0])

galchi_df.plot(color='blue',ax=ax[1])

suirenitar_df.plot(color='green',ax=ax[2])


Calculate Discharge

In [18]:
galchi_df['discharge']=galchi_df.apply(lambda row:calculate_discharge_galchi(water_level=row['value']),axis=1)
budhi_gandaki_df['discharge']=budhi_gandaki_df.apply(lambda row:calculate_discharge_budhi(water_level=row['value']),axis=1)
suirenitar_df['discharge']=suirenitar_df.apply(lambda row:calculate_discharge_suirenitar(water_level=row['value']),axis=1)

In [21]:
galchi_df['mean_velocity']=galchi_df.apply(lambda row:calculate_mean_velocity_galchi(row['discharge']),axis=1)
budhi_gandaki_df['mean_velocity']=budhi_gandaki_df.apply(lambda row:calculate_mean_velocity_buddhi(row['discharge']),axis=1)

In [ ]:
galchi_df[['time_lag_hr', 'time_lag_min']] = galchi_df.apply(
    lambda row: pd.Series(calcualte_time(DISTANCE_GALCHI_SUIRENITAR, row['mean_velocity'])), axis=1
)
budhi_df[['time_lag_hr', 'time_lag_min']] = budhi_df.apply(
    lambda row: pd.Series(calcualte_time(DISTANCE_BUDHI_SUIRENITAR, row['mean_velocity'])), axis=1
)
